In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import osu.dataset as dataset


osu! path: C:\Users\niooi\AppData\Roaming/../Local/osu!


In [2]:
# reload from saved

xs = np.load(f'.datasets/xs_7500_07-30_22-38-46.npy')
ys = np.load(f'.datasets/ys_7500_07-30_22-38-46.npy')

# xs = np.load(f'.datasets/xs_142_07-21_21-50-55.npy')
# ys = np.load(f'.datasets/ys_142_07-21_21-50-55.npy')

In [3]:
xs.shape

(26648, 2048, 9)

In [4]:
import torch

BATCH_SIZE = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

20

In [22]:
from models.vae import OsuReplayVAE
from models.annealer import Annealer

annealer = Annealer(
    total_steps=10,
    range=(0, 0.38),
    cyclical=True,
    stay_max_steps=5
)

vae = OsuReplayVAE(batch_size=BATCH_SIZE, annealer=annealer, noise_std=0.045)

# vae = OsuReplayVAE.load(".trained/vae_353rcl_9ft.pt")

vae.load_data(xs, ys)

VAE models compiled..
VAE Models initialized on cuda (noise_std=0.045)
Encoder parameters: 235296
Decoder parameters: 148082
Data loaded: 21318 training samples, 5330 test samples (position only)


In [23]:
from models.keys import OsuKeyModel

keys = OsuKeyModel(batch_size=BATCH_SIZE)
keys.load_data(xs, ys)

Key Model initialized on cuda (noise_std=0.0)
Key model parameters: 54626
Data loaded: 21318 training samples, 5330 test samples (keys only)


In [24]:
model = vae

EPOCHS = 3

for i in range(25):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

Epoch 1/3 (Beta: 1000.0): 100%|██████████| 334/334 [00:33<00:00, 10.00it/s]


Epoch 1/3, Total: 3486.9151, Recon: 3447.8382, KL: 0.0391


Epoch 2/3 (Beta: 1100.044): 100%|██████████| 334/334 [00:41<00:00,  8.08it/s]


Epoch 2/3, Total: 1050.8884, Recon: 1050.5219, KL: 0.0003


Epoch 3/3 (Beta: 1200.088): 100%|██████████| 334/334 [00:42<00:00,  7.92it/s]


Epoch 3/3, Total: 778.8285, Recon: 778.7048, KL: 0.0001
VAE models saved to .trained/vae_2025-07-31_01-01-39.pt


Epoch 1/3 (Beta: 1300.132): 100%|██████████| 334/334 [00:33<00:00, 10.00it/s]


Epoch 1/3, Total: 702.9846, Recon: 702.9329, KL: 0.0000


Epoch 2/3 (Beta: 1400.176):  91%|█████████ | 303/334 [00:27<00:02, 10.98it/s]


KeyboardInterrupt: 

In [ ]:
for i in range(300):
    model.train(EPOCHS)
    model.save()
    
model.plot_losses()

In [17]:
# test training results
from osu.rulesets.mods import Mods
import osu.rulesets.beatmap as bm
import osu.dataset as dataset

test_name = 'ringyou'
test_mods = 0
test_map_path = f'assets/{test_name}_map.osu'
test_song = f'assets/{test_name}_song.mp3'

test_map = bm.load(test_map_path)
test_map.apply_mods(test_mods)

data = dataset.input_data(test_map)
data = np.reshape(data.values, (-1, dataset.BATCH_LENGTH, len(dataset.INPUT_FEATURES)))
data = torch.FloatTensor(data)

Turning beatmaps into time series data: 100%|██████████| 1/1 [00:00<00:00,  3.91it/s]


In [18]:
replay_data = vae.generate(data)
    
replay_data = np.concatenate(replay_data)
replay_data = np.pad(replay_data, ((0, 0), (0, 2)), mode='constant', constant_values=0)
if not os.path.exists('.generated'):
    os.makedirs('.generated')
    
print(f"Generated replay data shape: {replay_data.shape}")


Generated replay data shape: (12288, 4)


In [19]:
import osu.preview.preview as preview

preview.preview_replay_raw(replay_data, test_map_path, test_mods, test_song)